In [8]:
import pandas as pd
import numpy as np
import os
import pathlib
import joblib
from fredapi import Fred
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import shap
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import json


# Load FRED API key from .env file
env_path = pathlib.Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)

# Define series to fetch from FRED
series_dict = {
    'gdp': ('GDP', 'Q'),
    'cpi': ('CPIAUCSL', 'M'),
    'unemp': ('UNRATE', 'M'),
    'fedrate': ('FEDFUNDS', 'M'),
    'bonds2y': ('DGS2', 'D'),
    'bonds10y': ('DGS10', 'D'),
}

# Fetch data from FRED
data = pd.DataFrame()
for var_name, (series_id, freq) in series_dict.items():
    series_data = fred.get_series(series_id)
    data[var_name] = series_data

# Resample monthly data to quarterly and calculate additional variables
data_q = pd.DataFrame()
data_q['fedrate'] = data['fedrate'].resample('Q').mean()
data_q['gdp'] = data['gdp'].resample('Q').last()
data_q['unemp'] = data['unemp'].resample('Q').mean()
data_q['cpi'] = data['cpi'].resample('Q').last()
data_q['cpi_pct'] = data_q['cpi'].pct_change() * 100
data_q['gdp_pct'] = data_q['gdp'].pct_change() * 100
data_q['gdp_pct_ma4'] = data_q['gdp_pct'].rolling(window=4).mean()

# Define start and end dates
start_date = '1955-01-01'
end_date = '2022-12-31'

# Filter data based on specified date range
data_input = data_q[['fedrate', 'gdp', 'gdp_pct', 'gdp_pct_ma4', 'cpi', 'cpi_pct', 'unemp']].copy()
data_input['fedrate_change'] = data_input['fedrate'].diff()
data_input['target'] = (data_input['fedrate_change'] > 0).astype(int)
data_input['cpi'] = data_input['cpi_pct']

# Extend the start date to include additional rows for lag features
extended_start_date = pd.to_datetime(start_date) - pd.DateOffset(months=6)  # extend by two quarters
data_input = data_input.loc[extended_start_date:end_date]

# Apply lag operations
# data_input['cpi_pct_lag1'] = data_input['cpi_pct'].shift(1)
# data_input['cpi_pct_lag2'] = data_input['cpi_pct'].shift(2)
# data_input['unrate_lag1'] = data_input['unrate'].shift(1)
# data_input['unrate_lag2'] = data_input['unrate'].shift(2)
# data_input['mult'] = data_input['cpi_pct'] * data_input['unrate']
# data_input['mult_lag1'] = data_input['mult'].shift(1)

# Filter the data again to the desired date range
df = data_input.loc[start_date:end_date]
df = df[['target',  'cpi', 'unemp',  'fedrate_change', 'fedrate']]
df

,target,cpi,unemp,fedrate_change,fedrate
1955-03-31,1,0.187126,4.9,0.54,1.39
1955-06-30,1,0.074710,4.7,0.04,1.43
1955-09-30,1,-0.111982,4.0,0.25,1.68
1955-12-31,1,0.224215,4.3,0.56,2.24
1956-03-31,1,0.037286,4.0,0.21,2.45
...,...,...,...,...,...
2021-12-31,0,1.750784,4.5,-0.02,0.08
2022-03-31,0,2.197655,4.0,0.00,0.08
2022-06-30,1,2.127396,3.6,0.25,0.33
2022-09-30,1,2.084813,3.5,1.35,1.68


In [9]:
# Preprocessed data
X = df[['unemp', 'cpi']]
y = df['target']

# Split the dataset while preserving time order
train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add a constant to the features
X_train_scaled_sm = sm.add_constant(X_train_scaled)
X_test_scaled_sm = sm.add_constant(X_test_scaled)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train_scaled_sm).fit()
print(logit_model.summary())

# Get the predicted probabilities
log_reg_probs = logit_model.predict(X_test_scaled_sm)

# Compute ROC Curve and AUC Score
fpr, tpr, thresholds = roc_curve(y_test, log_reg_probs)
roc_auc = auc(fpr, tpr)

# Plotting ROC Curve
roc_fig = px.line(
    x=fpr, y=tpr,
    labels={'x': 'False Positive Rate', 'y': 'True Positive Rate'},
    title='Receiver Operating Characteristic',
    width=600, height=600
)
roc_fig.add_shape(
    type='line', line=dict(dash='dash', color='navy'),
    x0=0, x1=1, y0=0, y1=1
)
roc_fig.add_annotation(
    x=0, y=1,
    text=f'AUC: {roc_auc:.2f}',
    showarrow=False
)
roc_fig.write_image('roc_curve.png', width=600, height=600)  # Save as PNG

# Get the predicted classes
log_reg_preds = np.round(log_reg_probs)

# Print Classification Report
print("Logistic Regression Classification Report:")
print(classification_report(y_test, log_reg_preds))

Optimization terminated successfully.
         Current function value: 0.666396
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  217
Model:                          Logit   Df Residuals:                      214
Method:                           MLE   Df Model:                            2
Date:                Wed, 01 Nov 2023   Pseudo R-squ.:                 0.03610
Time:                        11:46:45   Log-Likelihood:                -144.61
converged:                       True   LL-Null:                       -150.02
Covariance Type:            nonrobust   LLR p-value:                  0.004447
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1216      0.139      0.872      0.383      -0.152       0.395
x1            -0.4176      0.

In [3]:
joblib.dump(logit_model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [5]:
# Load the model and scaler
loaded_model = joblib.load('fed_model.pkl')
loaded_scaler = joblib.load('fed_scaler.pkl')

# New data point
new_data = np.array([[3.8, 0.8003404592684449]])

# Transform the new data using the loaded scaler
new_data_scaled = loaded_scaler.transform(new_data)

# Add a constant to the scaled data (ensure it's the first column)
new_data_scaled_sm = sm.add_constant(new_data_scaled, has_constant='add')

# Use the loaded model to predict the outcome probabilities
outcome_probs = loaded_model.predict(new_data_scaled_sm)

print("Outcome probabilities:", outcome_probs)


Outcome probabilities: [0.65703603]


C:\Users\Maciek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

